In [1]:
"""
This is a program to create "diagnostic plots" from data files that have already been separated by filter and/or date.

Author: Kurt Ger '23

Versions:
1.0: Creates correct (but hideous) plots of data from day-separated ATLAS data files.
1.0a: Corrected for use on clean datafiles. Docstrings added.
"""

'\nThis is a program to create "diagnostic plots" from data files that have already been separated by filter and/or date.\n\nAuthor: Kurt Ger \'23\n\nVersions:\n1.0: Creates correct (but hideous) plots of data from day-separated ATLAS data files.\n1.0a: Corrected for use on clean datafiles. Docstrings added.\n'

In [2]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import tkinter.ttk as ttk
import os

import matplotlib.pyplot as plt
import numpy as np

In [3]:
def dailyPlot(catapult, tarmac):
    """
    Creates JD-by-JD graphs.
    Algorithm:
    -Obtain list of files, distinguish CSVs
    -For each CSV, read in columns of date and magnitude
    -Create plot, save as .png
    """
    files = os.listdir(catapult)

    for file in files:
        if file[-4:] != ".csv":     #this way, non-csv are ignored
            continue

        source = catapult + "/" + file
        template = open(source, "r")
        nameOnly = source.split("/")
        response = tarmac + "/" + nameOnly[-1][:-4] + ".png"

        inlines = template.readlines()
        outlines = []

        xVals = []
        yVals = []
        xErr = []
        yErr = []

        for line in inlines:
            outlines.append(line.split(","))
        outlines = outlines[1:]

        for line in outlines:
            xVals.append(float(line[0]))
            yVals.append(float(line[2]))
            xErr.append(float(line[7]))
            yErr.append(float(line[3]))

        xVals = np.array(xVals)
        #print(xVals)
        yVals = np.array(yVals)
        #print(yVals)

        plt.plot(xVals, yVals, 'o')
        plt.xlabel("JD-2459000")
        plt.ylabel("Magnitude")
        plt.errorbar(xVals, yVals, xerr = xErr, yerr = yErr, fmt ='o')

        plt.savefig(response)
        plt.clf()
        plt.cla()           #i don't know why, but in the absence of these two lines, every day is mushed together
        template.close()

    messagebox.showinfo("Finished","Daily plotting complete. Check destination folder.")

In [4]:
def main():
    try:
        appWindow = tk.Tk()


        takeoff = filedialog.askdirectory(parent=appWindow,
                                 initialdir=os.getcwd(),
                                 title="Please select source folder:")


        landing = filedialog.askdirectory(parent=appWindow,
                                 initialdir=os.getcwd(),
                                 title="Please select destination folder:")


        #print(template)
        #print(landing)

        dailyPlot(takeoff, landing)
        appWindow.destroy()

    except FileNotFoundError:
        messagebox.showerror("Error", "Specified directory does not exist. Exiting.")
        appWindow.destroy()

In [5]:
main()